# 2. Classification

*(Modified version of lession 2 in https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/)*
<br/><br/>

## Setup

### Install dependencies

In [ ]:
%pip install python-dotenv~=1.0 docarray~=0.40.0 pydantic~=2.9 pypdf~=5.1 --upgrade --quiet
%pip install openai~=1.54 --upgrade --quiet

# If running locally, you can do this instead:
#%pip install -r ../requirements.txt

#### Load environment variables

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# If running in Google Colab, you can use this code instead:
# from google.colab import userdata
# os.environ["AZURE_OPENAI_API_KEY"] = userdata.get("AZURE_OPENAI_API_KEY")
# os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get("AZURE_OPENAI_ENDPOINT")

#### Setup OpenAI Client and helper function

In [ ]:
from openai import AzureOpenAI

# Create a an OpenAI client, connecting to OpenAI via an Azure deployment 
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# These models (deployments) are currently available: gpt-4o, gpt-4o-mini, o1-mini, o1-preview  
def get_completion(prompt: str=None, model: str = "gpt-4o", messages: list = None, useJson: bool = False) -> str:
    if messages is None:
        messages = [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.1, # 0.0: deterministic, 1.0: more creative
        response_format={ "type": "json_object" } if useJson else { "type": "text" },
    )
    return response.choices[0].message.content

#### Classify customer queries to handle different cases

In [ ]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Classify each query into a primary category \
and a secondary category. 
Provide your output in json format with the \
keys: primary and secondary.

Primary categories: Billing, Technical Support, \
Account Management, or General Inquiry.

Billing secondary categories:
Unsubscribe or upgrade
Add a payment method
Explanation for charge
Dispute a charge

Technical Support secondary categories:
General troubleshooting
Device compatibility
Software updates

Account Management secondary categories:
Password reset
Update personal information
Close account
Account security

General Inquiry secondary categories:
Product information
Pricing
Feedback
Speak to a human

"""
user_message = f"""\
I want you to delete my profile and all of my user data"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion(messages=messages, useJson=True)
print(response)

In [ ]:
user_message = f"""\
Tell me more about your flat screen tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion(messages=messages, useJson=True)
print(response)